### NLP Assignment-2

Load data

In [ ]:
!wget https://raw.githubusercontent.com/debajyotimaz/nlp_assignment/main/train_split.csv
!wget https://raw.githubusercontent.com/debajyotimaz/nlp_assignment/main/test_split.csv

--2024-10-17 17:41:13--  https://raw.githubusercontent.com/debajyotimaz/nlp_assignment/main/train_split.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 144474 (141K) [text/plain]
Saving to: ‘train_split.csv.1’

train_split.csv.1   100%[===================>] 141.09K  --.-KB/s    in 0.03s   

2024-10-17 17:41:13 (4.38 MB/s) - ‘train_split.csv.1’ saved [144474/144474]

--2024-10-17 17:41:13--  https://raw.githubusercontent.com/debajyotimaz/nlp_assignment/main/test_split.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 35259 (3

In [ ]:
import numpy as np
import pandas as pd
import re
from matplotlib import pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import roc_auc_score, accuracy_score, roc_curve, confusion_matrix, \
multilabel_confusion_matrix, classification_report
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier

import warnings
warnings.filterwarnings('ignore')

In [ ]:
import pandas as pd

# Load the training DataFrame
train_data = pd.read_csv('/content/train_split.csv')
train_data.head()

,text,Joy,Fear,Anger,Sadness,Surprise
0,The light would come all the way up to your ve...,0,1,0,0,1
1,"Well, my birthday is in January.",0,0,0,0,0
2,"Tears in my eyes, too much on my mind, and dee...",0,1,0,1,0
3,"My eyes scanned quickly into every room, every...",0,1,1,0,0
4,I felt the afterglow of the late afternoon sun...,1,0,0,0,0


In [ ]:
train_data_set=train_data["text"]
train_data_set.head()

,text
0,The light would come all the way up to your ve...
1,"Well, my birthday is in January."
2,"Tears in my eyes, too much on my mind, and dee..."
3,"My eyes scanned quickly into every room, every..."
4,I felt the afterglow of the late afternoon sun...


In [ ]:
train_labels=train_data[train_data.columns[1:]]
train_labels.head()

,Joy,Fear,Anger,Sadness,Surprise
0,0,1,0,0,1
1,0,0,0,0,0
2,0,1,0,1,0
3,0,1,1,0,0
4,1,0,0,0,0


In [ ]:
!pip install nltk

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
from collections import defaultdict
import gc

def load_data(csv_file):
    df = pd.read_csv(csv_file)
    return df

def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    tokens = word_tokenize(text)
    return tokens

csv_file_path = '/content/train_split.csv'
data = load_data(csv_file_path)

# Preprocess text data
data['processed_text'] = data['text'].apply(preprocess_text)

# Prepare the labels (multi-label binary encoding)
labels = data[['Joy', 'Fear', 'Anger', 'Sadness', 'Surprise']].values

# Build vocabulary
word_counts = defaultdict(int)
for doc in data['processed_text']:
    for word in doc:
        word_counts[word] += 1

vocab = list(word_counts.keys())
word2index = {word: idx for idx, word in enumerate(vocab)}
index2word = {idx: word for word, idx in word2index.items()}

# Cbow model
def generate_cbow_training_data(processed_documents, window_size):
    training_data = []
    for sentence in processed_documents:
        for idx, target_word in enumerate(sentence):
            if target_word not in word2index:
                continue
            target_word_idx = word2index[target_word]
            context = []
            for neighbor in range(max(idx - window_size, 0), min(idx + window_size + 1, len(sentence))):
                if neighbor != idx and sentence[neighbor] in word2index:
                    context_word_idx = word2index[sentence[neighbor]]
                    context.append(context_word_idx)
            if len(context) > 0:
                training_data.append((context, target_word_idx))
    return training_data

window_size = 4
training_data = generate_cbow_training_data(data['processed_text'], window_size)

# Initialize weights
embedding_size = 100
W1 = np.random.rand(len(vocab), embedding_size)
W2 = np.random.rand(embedding_size, len(vocab))

def softmax(x):
    exp_x = np.exp(x - np.max(x))
    return exp_x / exp_x.sum(axis=0)


def train_cbow(training_data, learning_rate, epochs, batch_size):
    global W1, W2
    for epoch in range(epochs):
        np.random.shuffle(training_data)
        total_loss = 0
        for i in range(0, len(training_data), batch_size):
            batch = training_data[i:i + batch_size]
            W1_grad = np.zeros_like(W1)
            W2_grad = np.zeros_like(W2)

            for context, target_word_idx in batch:
                h = np.mean(W1[context], axis=0)
                u = np.dot(W2.T, h)
                y_pred = softmax(u)

                total_loss -= np.log(y_pred[target_word_idx])

                y_true = np.zeros(len(vocab))
                y_true[target_word_idx] = 1
                e = y_pred - y_true

                for context_word_idx in context:
                    W1_grad[context_word_idx] += np.dot(W2, e)
                W2_grad += np.outer(h, e)

            W1 -= learning_rate * W1_grad / batch_size
            W2 -= learning_rate * W2_grad / batch_size

        average_loss = total_loss / len(training_data)
        scaled_loss = average_loss / 100

        print(f'Epoch {epoch + 1}, Loss: {scaled_loss:.4f}')


# Train CBOW model
'''learning_rate = 0.001
epochs = 20
batch_size = 64
train_cbow(training_data, learning_rate, epochs, batch_size)

# Prepare input features using the word embeddings
def get_average_embedding(text):
    tokens = preprocess_text(text)
    vectors = []
    for token in tokens:
        if token in word2index:
            vectors.append(W1[word2index[token]])
    if vectors:
        return np.mean(vectors, axis=0)
    else:
        return np.zeros(embedding_size)

# Compute embeddings for all texts and save them to a text file
embeddings = []
for text in data['text']:
    embedding = get_average_embedding(text)
    embeddings.append(embedding)

# Save embeddings to a text file
with open('text_embeddings.txt', 'w') as f:
    for text, embedding in zip(data['text'], embeddings):
        embedding_str = ' '.join(map(str, embedding))
        f.write(f"{text}\t{embedding_str}\n")

print("Embeddings saved to text_embeddings.txt")'''


'learning_rate = 0.001\nepochs = 20\nbatch_size = 64\ntrain_cbow(training_data, learning_rate, epochs, batch_size)\n\n# Prepare input features using the word embeddings\ndef get_average_embedding(text):\n    tokens = preprocess_text(text)\n    vectors = []\n    for token in tokens:\n        if token in word2index:\n            vectors.append(W1[word2index[token]])\n    if vectors:\n        return np.mean(vectors, axis=0)\n    else:\n        return np.zeros(embedding_size)\n\n# Compute embeddings for all texts and save them to a text file\nembeddings = []\nfor text in data[\'text\']:\n    embedding = get_average_embedding(text)\n    embeddings.append(embedding)\n\n# Save embeddings to a text file\nwith open(\'text_embeddings.txt\', \'w\') as f:\n    for text, embedding in zip(data[\'text\'], embeddings):\n        embedding_str = \' \'.join(map(str, embedding))\n        f.write(f"{text}\t{embedding_str}\n")\n\nprint("Embeddings saved to text_embeddings.txt")'

In [ ]:
!wget https://raw.githubusercontent.com/ashishkumar0803/21058_Ashish_Kumar_nlpassignment2/refs/heads/main/text_embeddings_final2.txt

--2024-10-17 17:41:17--  https://raw.githubusercontent.com/ashishkumar0803/21058_Ashish_Kumar_nlpassignment2/refs/heads/main/text_embeddings_final2.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3192105 (3.0M) [text/plain]
Saving to: ‘text_embeddings_final2.txt.1’

text_embeddings_fin 100%[===================>]   3.04M  --.-KB/s    in 0.09s   

2024-10-17 17:41:18 (33.1 MB/s) - ‘text_embeddings_final2.txt.1’ saved [3192105/3192105]



In [ ]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from sklearn.metrics import classification_report


In [ ]:
def load_embeddings(file_path):
    texts = []
    embeddings = []
    with open(file_path, 'r') as f:
        for line in f:
            parts = line.strip().split('\t')
            texts.append(parts[0])  # The original text
            embedding = list(map(float, parts[1].split()))
            embeddings.append(embedding)
    return texts, np.array(embeddings)

file_path = '/content/text_embeddings_final2.txt'

texts, embeddings = load_embeddings(file_path)

labels_df = train_data[train_data.columns[1:]]
labels = labels_df[['Joy', 'Fear', 'Anger', 'Sadness', 'Surprise']].values

In [ ]:
# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(embeddings, labels, test_size=0.2, random_state=42)

In [ ]:
# FFNN model
model = Sequential()
model.add(Dense(64, activation='relu', input_dim=X_train.shape[1]))
model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(5, activation='sigmoid'))

model.compile(optimizer='AdamW', loss='binary_crossentropy', metrics=['accuracy'])

model.fit(X_train, y_train, epochs=200, batch_size=64, validation_data=(X_val, y_val))

y_pred_ffnn = model.predict(X_val)
y_pred_ffnn = (y_pred_ffnn > 0.5).astype(int) # Convert probabilities to binary (0 or 1)

# classification report for the FFNN model
report_ffnn = classification_report(y_val, y_pred_ffnn, target_names=['Joy', 'Fear', 'Anger', 'Sadness', 'Surprise'], zero_division=0)
print("Classification Report for FFNN Model:\n", report_ffnn)

Epoch 1/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 41s 14ms/step - accuracy: 0.0581 - loss: 0.6912 - val_accuracy: 0.0406 - val_loss: 0.6268
Epoch 2/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.2522 - loss: 0.6085 - val_accuracy: 0.5594 - val_loss: 0.5719
Epoch 3/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5378 - loss: 0.5779 - val_accuracy: 0.5594 - val_loss: 0.5632
Epoch 4/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5305 - loss: 0.5743 - val_accuracy: 0.5594 - val_loss: 0.5620
Epoch 5/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5214 - loss: 0.5671 - val_accuracy: 0.5594 - val_loss: 0.5614
Epoch 6/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5526 - loss: 0.5699 - val_accuracy: 0.5594 - val_loss: 0.5628
Epoch 7/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.5492 - loss: 0.5674 - val_accuracy: 0.5594 - val_loss: 0.5611
Epoch 8/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.5444 - loss: 0.5670 - val_accuracy: 0.5594 -

In [ ]:
X_train_rnn = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_val_rnn = X_val.reshape((X_val.shape[0], 1, X_val.shape[1]))

# LSTM model
model = Sequential()
model.add(LSTM(64, input_shape=(128, X_train.shape[1])))
model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(5, activation='sigmoid'))

model.compile(optimizer='AdamW', loss='binary_crossentropy', metrics=['accuracy'])

model.fit(X_train_rnn, y_train, epochs=200, batch_size=64, validation_data=(X_val_rnn, y_val))

y_pred_lstm = model.predict(X_val_rnn)
y_pred_lstm = (y_pred_lstm > 0.5).astype(int)  # Convert probabilities to binary (0 or 1)

# classification report for the LSTM model
report_lstm = classification_report(y_val, y_pred_lstm, target_names=['Joy', 'Fear', 'Anger', 'Sadness', 'Surprise'], zero_division=0)
print("Classification Report for LSTM Model:\n", report_lstm)

Epoch 1/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 3s 22ms/step - accuracy: 0.5289 - loss: 0.6349 - val_accuracy: 0.5594 - val_loss: 0.5709
Epoch 2/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.5475 - loss: 0.5804 - val_accuracy: 0.5594 - val_loss: 0.5653
Epoch 3/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.5312 - loss: 0.5733 - val_accuracy: 0.5594 - val_loss: 0.5637
Epoch 4/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.5594 - loss: 0.5624 - val_accuracy: 0.5594 - val_loss: 0.5641
Epoch 5/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5402 - loss: 0.5718 - val_accuracy: 0.5594 - val_loss: 0.5618
Epoch 6/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5368 - loss: 0.5765 - val_accuracy: 0.5594 - val_loss: 0.5613
Epoch 7/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.5323 - loss: 0.5691 - val_accuracy: 0.5594 - val_loss: 0.5614
Epoch 8/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.5340 - loss: 0.5781 - val_accuracy: 0.5594 - 

In [ ]:
from sklearn.metrics import classification_report
import numpy as np

# Example of gold labels (y_true) and predictions (y_pred)
y_true = np.array([[1, 0, 0, 1, 0],  # joy, sadness, fear, anger, surprise
                   [0, 1, 1, 0, 0],
                   [1, 0, 1, 0, 1]])

y_pred = np.array([[1, 0, 0, 1, 0],  # joy, sadness, fear, anger, surprise
                   [0, 1, 0, 1, 0],
                   [1, 0, 1, 0, 0]])

# Generate the classification report
report = classification_report(y_true, y_pred, target_names=['joy', 'sadness', 'fear', 'anger', 'surprise'], zero_division=0)

print(report)

              precision    recall  f1-score   support

         joy       1.00      1.00      1.00         2
     sadness       1.00      1.00      1.00         1
        fear       1.00      0.50      0.67         2
       anger       0.50      1.00      0.67         1
    surprise       0.00      0.00      0.00         1

   micro avg       0.83      0.71      0.77         7
   macro avg       0.70      0.70      0.67         7
weighted avg       0.79      0.71      0.71         7
 samples avg       0.83      0.72      0.77         7

